In [ ]:
from pushcart.metadata import Metadata
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

# Working with flat data

In [12]:
input_flat_data = (
    spark.read.format("csv")
    .option("header", "true")
    .option("escape", '"')
    .load("tests/data/sample_data.csv")
)

print(input_flat_data.schema)  # noqa: T201
input_flat_data.pandas_api()

StructType([StructField('id', StringType(), True), StructField('ts', StringType(), True), StructField('payload', StringType(), True)])


,id,ts,payload
0,0,2023-07-13T17:26:59.345122,"{""current_page"": 1, ""data"": [{""fact"": ""Unlike ..."
1,1,2023-07-13T17:27:01.746471,"{""current_page"": 2, ""data"": [{""fact"": ""During ..."
2,2,2023-07-13T17:27:05.066221,"{""current_page"": 3, ""data"": [{""fact"": ""A cat c..."
3,3,2023-07-13T17:27:07.392719,"{""current_page"": 4, ""data"": [{""fact"": ""A cat\u..."
4,4,2023-07-13T17:27:12.699642,"{""current_page"": 5, ""data"": [{""fact"": ""A cat c..."
5,5,2023-07-13T17:27:16.992408,"{""current_page"": 6, ""data"": [{""fact"": ""In the ..."
6,6,2023-07-13T17:27:19.372787,"{""current_page"": 7, ""data"": [{""fact"": ""All cat..."
7,7,2023-07-13T17:27:21.721222,"{""current_page"": 8, ""data"": [{""fact"": ""Approxi..."
8,8,2023-07-13T17:27:26.052877,"{""current_page"": 9, ""data"": [{""fact"": ""A cat\u..."


In [6]:
md = Metadata(input_flat_data, infer_fraction=1.0)
md.get_metadata()

2024-05-16 10:06:21.056 | INFO     | pushcart.metadata.metadata:_infer_timestamps:57 - Attempting to infer timestamp format for id column.
2024-05-16 10:06:21.149 | WARNING  | pushcart.metadata.metadata:_infer_timestamps:69 - Could not infer timestamp format for id column.
2024-05-16 10:06:21.149 | INFO     | pushcart.metadata.metadata:_infer_json_schema:36 - Attempting to infer JSON schema for id column.
2024-05-16 10:06:21.312 | WARNING  | pushcart.metadata.metadata:_infer_json_schema:43 - Could not infer JSON schema for id column.
2024-05-16 10:06:21.315 | INFO     | pushcart.metadata.metadata:_infer_timestamps:57 - Attempting to infer timestamp format for ts column.
2024-05-16 10:06:21.413 | INFO     | pushcart.metadata.metadata:_infer_timestamps:57 - Attempting to infer timestamp format for payload column.
2024-05-16 10:06:21.489 | WARNING  | pushcart.metadata.metadata:_infer_timestamps:69 - Could not infer timestamp format for payload column.
2024-05-16 10:06:21.490 | INFO     | 

In [5]:
# Edits are reflected in the underlying dataset
md.metadata_df

,column_order,source_column_name,source_column_type,dest_column_name,dest_column_type,transform_function,default_value,validation_rule,validation_action
0,0,id,string,id,string,,,,
1,1,ts,string,ts,string,"F.to_timestamp(F.col(""ts""), ""yyyy-MM-dd'T'HH:m...",,,
2,2,payload,string,payload,string,"F.from_json(F.col(""payload""), schema=""struct<c...",,,


In [13]:
transformed_data = md.transform()

print(transformed_data.schema)  # noqa: T201
transformed_data.pandas_api()

2024-05-16 10:21:25.663 | INFO     | pushcart.metadata.metadata:_drop_technical_cols:466 - Excluding technical columns: None


StructType([StructField('id', StringType(), True), StructField('ts', TimestampType(), True), StructField('payload', StructType([StructField('current_page', LongType(), True), StructField('data', ArrayType(StructType([StructField('fact', StringType(), True), StructField('length', LongType(), True)]), True), True), StructField('first_page_url', StringType(), True), StructField('from', LongType(), True), StructField('last_page', LongType(), True), StructField('last_page_url', StringType(), True), StructField('links', ArrayType(StructType([StructField('active', BooleanType(), True), StructField('label', StringType(), True), StructField('url', StringType(), True)]), True), True), StructField('next_page_url', StringType(), True), StructField('path', StringType(), True), StructField('per_page', LongType(), True), StructField('prev_page_url', StringType(), True), StructField('to', LongType(), True), StructField('total', LongType(), True)]), True)])


,id,ts,payload
0,0,2023-07-13 17:26:59.345122,"(1, [(Unlike dogs, cats do not have a sweet to..."
1,1,2023-07-13 17:27:01.746471,"(2, [(During the time of the Spanish Inquisiti..."
2,2,2023-07-13 17:27:05.066221,"(3, [(A cat can travel at a top speed of appro..."
3,3,2023-07-13 17:27:07.392719,"(4, [(A cat’s brain is biologically more simil..."
4,4,2023-07-13 17:27:12.699642,"(5, [(A cat can jump up to five times its own ..."
5,5,2023-07-13 17:27:16.992408,"(6, [(In the original Italian version of Cinde..."
6,6,2023-07-13 17:27:19.372787,"(7, [(All cats have claws, and all except the ..."
7,7,2023-07-13 17:27:21.721222,"(8, [(Approximately 1/3 of cat owners think th..."
8,8,2023-07-13 17:27:26.052877,"(9, [(A cat’s nose pad is ridged with a unique..."


In [11]:
code = md.generate_code()

2024-05-16 10:12:16.493 | INFO     | pushcart.metadata.metadata:_drop_technical_cols:466 - Excluding technical columns: None
2024-05-16 10:12:16.495 | INFO     | pushcart.metadata.spark:generate_code:130 - 
df = (df
	.withColumn("id", F.col("id"))
	.withColumn("ts", F.to_timestamp(F.col("ts"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS"))
	.withColumn("payload", F.from_json(F.col("payload"), schema="struct<current_page:bigint,data:array<struct<fact:string,length:bigint>>,first_page_url:string,from:bigint,last_page:bigint,last_page_url:string,links:array<struct<active:boolean,label:string,url:string>>,next_page_url:string,path:string,per_page:bigint,prev_page_url:string,to:bigint,total:bigint>"))
	.select(['id', 'ts', 'payload']))


# Working with nested data

In [18]:
input_nested_data = spark.read.format("parquet").load(
    "tests/data/sample_nested_data.parquet",
)

print(input_nested_data.schema)  # noqa: T201
input_nested_data.pandas_api()

StructType([StructField('id', StringType(), True), StructField('ts', TimestampType(), True), StructField('payload', StructType([StructField('current_page', LongType(), True), StructField('data', ArrayType(StructType([StructField('fact', StringType(), True), StructField('length', LongType(), True)]), True), True), StructField('first_page_url', StringType(), True), StructField('from', LongType(), True), StructField('last_page', LongType(), True), StructField('last_page_url', StringType(), True), StructField('links', ArrayType(StructType([StructField('active', BooleanType(), True), StructField('label', StringType(), True), StructField('url', StringType(), True)]), True), True), StructField('next_page_url', StringType(), True), StructField('path', StringType(), True), StructField('per_page', LongType(), True), StructField('prev_page_url', StringType(), True), StructField('to', LongType(), True), StructField('total', LongType(), True)]), True)])


,id,ts,payload
0,0,2023-07-13 17:26:59.345122,"(1, [(Unlike dogs, cats do not have a sweet to..."
1,1,2023-07-13 17:27:01.746471,"(2, [(During the time of the Spanish Inquisiti..."
2,2,2023-07-13 17:27:05.066221,"(3, [(A cat can travel at a top speed of appro..."
3,3,2023-07-13 17:27:07.392719,"(4, [(A cat’s brain is biologically more simil..."
4,4,2023-07-13 17:27:12.699642,"(5, [(A cat can jump up to five times its own ..."
5,5,2023-07-13 17:27:16.992408,"(6, [(In the original Italian version of Cinde..."
6,6,2023-07-13 17:27:19.372787,"(7, [(All cats have claws, and all except the ..."
7,7,2023-07-13 17:27:21.721222,"(8, [(Approximately 1/3 of cat owners think th..."
8,8,2023-07-13 17:27:26.052877,"(9, [(A cat’s nose pad is ridged with a unique..."


In [19]:
nested_md = Metadata(input_nested_data, infer_fraction=1.0)
nested_md.get_metadata()

2024-05-16 10:44:11.132 | INFO     | pushcart.metadata.metadata:_infer_timestamps:57 - Attempting to infer timestamp format for id column.
2024-05-16 10:44:11.217 | WARNING  | pushcart.metadata.metadata:_infer_timestamps:69 - Could not infer timestamp format for id column.
2024-05-16 10:44:11.218 | INFO     | pushcart.metadata.metadata:_infer_json_schema:36 - Attempting to infer JSON schema for id column.
2024-05-16 10:44:11.382 | WARNING  | pushcart.metadata.metadata:_infer_json_schema:43 - Could not infer JSON schema for id column.
2024-05-16 10:44:11.384 | INFO     | pushcart.metadata.metadata:_infer_timestamps:57 - Attempting to infer timestamp format for payload.data.fact column.
2024-05-16 10:44:11.456 | WARNING  | pushcart.metadata.metadata:_infer_timestamps:69 - Could not infer timestamp format for payload.data.fact column.
2024-05-16 10:44:11.457 | INFO     | pushcart.metadata.metadata:_infer_json_schema:36 - Attempting to infer JSON schema for payload.data.fact column.
2024-0

In [22]:
transformed_flattened_data = nested_md.transform()

print(transformed_flattened_data.schema)  # noqa: T201
transformed_flattened_data.pandas_api()[:5]

2024-05-16 10:47:15.376 | INFO     | pushcart.metadata.metadata:_drop_technical_cols:466 - Excluding technical columns: None


StructType([StructField('id', StringType(), True), StructField('ts', TimestampType(), True), StructField('payload_current_page', LongType(), True), StructField('payload_data', StructType([StructField('fact', StringType(), True), StructField('length', LongType(), True)]), True), StructField('payload_data_fact', StringType(), True), StructField('payload_data_length', LongType(), True), StructField('payload_first_page_url', StringType(), True), StructField('payload_from', LongType(), True), StructField('payload_last_page', LongType(), True), StructField('payload_last_page_url', StringType(), True), StructField('payload_links', StructType([StructField('active', BooleanType(), True), StructField('label', StringType(), True), StructField('url', StringType(), True)]), True), StructField('payload_links_active', BooleanType(), True), StructField('payload_links_label', StringType(), True), StructField('payload_links_url', StringType(), True), StructField('payload_next_page_url', StringType(), Tr

,id,ts,payload_current_page,payload_data,payload_data_fact,payload_data_length,payload_first_page_url,payload_from,payload_last_page,payload_last_page_url,payload_links,payload_links_active,payload_links_label,payload_links_url,payload_next_page_url,payload_path,payload_per_page,payload_prev_page_url,payload_to,payload_total
0,0,2023-07-13 17:26:59.345122,1,"(Unlike dogs, cats do not have a sweet tooth. ...","Unlike dogs, cats do not have a sweet tooth. S...",114,https://catfact.ninja/facts?page=1,1,34,https://catfact.ninja/facts?page=34,"(False, Previous, None)",False,Previous,None,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts,10,None,10,332
1,0,2023-07-13 17:26:59.345122,1,"(Unlike dogs, cats do not have a sweet tooth. ...","Unlike dogs, cats do not have a sweet tooth. S...",114,https://catfact.ninja/facts?page=1,1,34,https://catfact.ninja/facts?page=34,"(True, 1, https://catfact.ninja/facts?page=1)",True,1,https://catfact.ninja/facts?page=1,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts,10,None,10,332
2,0,2023-07-13 17:26:59.345122,1,"(Unlike dogs, cats do not have a sweet tooth. ...","Unlike dogs, cats do not have a sweet tooth. S...",114,https://catfact.ninja/facts?page=1,1,34,https://catfact.ninja/facts?page=34,"(False, 2, https://catfact.ninja/facts?page=2)",False,2,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts,10,None,10,332
3,0,2023-07-13 17:26:59.345122,1,"(Unlike dogs, cats do not have a sweet tooth. ...","Unlike dogs, cats do not have a sweet tooth. S...",114,https://catfact.ninja/facts?page=1,1,34,https://catfact.ninja/facts?page=34,"(False, 3, https://catfact.ninja/facts?page=3)",False,3,https://catfact.ninja/facts?page=3,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts,10,None,10,332
4,0,2023-07-13 17:26:59.345122,1,"(Unlike dogs, cats do not have a sweet tooth. ...","Unlike dogs, cats do not have a sweet tooth. S...",114,https://catfact.ninja/facts?page=1,1,34,https://catfact.ninja/facts?page=34,"(False, 4, https://catfact.ninja/facts?page=4)",False,4,https://catfact.ninja/facts?page=4,https://catfact.ninja/facts?page=2,https://catfact.ninja/facts,10,None,10,332


In [20]:
flattened_code = nested_md.generate_code()

2024-05-16 10:46:13.529 | INFO     | pushcart.metadata.metadata:_drop_technical_cols:466 - Excluding technical columns: None
2024-05-16 10:46:13.537 | INFO     | pushcart.metadata.spark:generate_code:130 - 
df = (df
	.withColumn("id", F.col("id"))
	.withColumn("ts", F.col("ts"))
	.withColumn("payload_current_page", F.col("payload.current_page"))
	.withColumn("payload_data", F.explode("payload.data"))
	.withColumn("payload_data_fact", F.col("payload_data.fact"))
	.withColumn("payload_data_length", F.col("payload_data.length"))
	.withColumn("payload_first_page_url", F.col("payload.first_page_url"))
	.withColumn("payload_from", F.col("payload.from"))
	.withColumn("payload_last_page", F.col("payload.last_page"))
	.withColumn("payload_last_page_url", F.col("payload.last_page_url"))
	.withColumn("payload_links", F.explode("payload.links"))
	.withColumn("payload_links_active", F.col("payload_links.active"))
	.withColumn("payload_links_label", F.col("payload_links.label"))
	.withColumn("payloa